### Data수집 : 574개의 회사의 ESG grade, Return, Aggregate Value, Industry Sector

- ESG Grade : KCGS
- Industry Sector : 한국표준산업분류

- esg_final_data.xlsx

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import yfinance as yf

df = pd.read_excel("./esg_final_data.xlsx") 
Firm = list(df['company'])

### KCGS의 ESG등급 Ordinal Data를  Score로 변환 

박은진, <ESG전략이 기업의 재무 성과에 미치는 영향(2018)>, KAIST

- 2017년부터 S등급 신설 및 최대점수 150점
- A+:120점, D:70점

In [14]:
esg_score = list(df['esg'])
e_score = list(df['e'])
s_score = list(df['s'])
g_score = list(df['g'])

for i in range(len(Firm)):
    if esg_score[i]==6: esg_score[i]=120
    if esg_score[i]==5: esg_score[i]=110
    if esg_score[i]==4: esg_score[i]=100
    if esg_score[i]==3: esg_score[i]=90
    if esg_score[i]==2: esg_score[i]=80
    if esg_score[i]==1: esg_score[i]=70
    
    if e_score[i]==6: e_score[i]=120
    if e_score[i]==5: e_score[i]=110
    if e_score[i]==4: e_score[i]=100
    if e_score[i]==3: e_score[i]=90
    if e_score[i]==2: e_score[i]=80
    if e_score[i]==1: e_score[i]=70
        
    if s_score[i]==6: s_score[i]=120
    if s_score[i]==5: s_score[i]=110
    if s_score[i]==4: s_score[i]=100
    if s_score[i]==3: s_score[i]=90
    if s_score[i]==2: s_score[i]=80
    if s_score[i]==1: s_score[i]=70
        
    if g_score[i]==6: g_score[i]=120
    if g_score[i]==5: g_score[i]=110
    if g_score[i]==4: g_score[i]=100
    if g_score[i]==3: g_score[i]=90
    if g_score[i]==2: g_score[i]=80
    if g_score[i]==1: g_score[i]=70


In [15]:
ret = list(df['ret'])
AV = list(df['AV'])

### Score Conversion by Sector

##### - Kim, Li and Li(2014) 

- ESG score for firm i = (ESG for form i - Min ESG for firm i's industry)/(Max ESG for firm i's industry - Min ESG for firm i's industry)


- BUT.... 등급매기기 전의 원점수 수집 필요...  Further study point!

In [16]:
original_sector=list(df['sector_class']) 
sector = set(df['sector_class']) 
sector = list(sector)
print(sector)
print('\nNumber of Sector:',len(sector))

['사업시설 관리, 사업 지원 및 임대 서비스업', '제조업', '전기, 가스, 증기 및 공기 조절 공급업', '전문, 과학 및 기술 서비스업', '정보통신업', '운수 및 창고업', '도매 및 소매업', '금융 및 보험업', '건설업']

Number of Sector: 9


In [17]:
L=[]
for i in range(len(sector)):
    L.append([])
for i in range(len(Firm)):
    for j in range(len(sector)):
         if original_sector[i] == sector[j]:
            L[j].append(i)

score=[]
for i in range(len(sector)):
    score.append([])
for i in range(len(sector)):
    for j in L[i]:
        score[i].append(esg_score[j])
        
max_score=[]        
min_score=[]
for i in range(len(sector)):
    max_score.append(max(score[i]))
    min_score.append(min(score[i]))

convert_score=[]
for i in range(len(Firm)):
    for j in range(len(sector)):
        if original_sector[i] == sector[j]:
            try: 
                p = (esg_score[i]-min_score[j])/(max_score[j]-min_score[j])
            except:
                convert_score.append(esg_score[i]) 
            else:
                if esg_score[i]-min_score[j]==0:
                    convert_score.append(esg_score[i])
                else:
                    convert_score.append((esg_score[i]-min_score[j])/(max_score[j]-min_score[j]))

### Logistic Regression Model

In [18]:
sector_num=list(df['sector_num'])

In [30]:
import math as m

df = pd.read_excel("./esg_final_data.xlsx")
Firm = list(df['company'])
ret = list(df['ret'])
for i in range(len(Firm)):
    if ret[i]>0:
        ret[i]=1
    else:
        ret[i]=0
        
df2=pd.DataFrame()
df2['ret']=ret
df2['esg']=esg_score
df2['e']=e_score
df2['s']=s_score
df2['g']=g_score
df2['AV']=AV
#df2['sector_num']=sector_num

log_AV=[]
for i in range(len(Firm)):
    log_AV.append(m.log(AV[i]))

df2['log_AV']=log_AV

In [31]:
df2.head()

,ret,esg,e,s,g,AV,log_AV
0,0,120,100,120,120,1.851312e+12,28.246916
1,1,120,110,120,120,1.148500e+12,27.769478
2,1,120,110,110,120,1.110984e+12,27.736267
3,1,120,120,120,120,1.804606e+13,30.523949
4,0,120,110,120,120,7.790729e+12,29.683956


In [25]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

df2.logit=smf.glm('ret ~ e+s+g', data=df2, family=sm.families.Binomial()).fit()
df2.logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    ret   No. Observations:                  561
Model:                            GLM   Df Residuals:                      557
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -302.33
Date:                Thu, 20 May 2021   Deviance:                       604.65
Time:                        21:57:11   Pearson chi2:                     563.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8132      1.242      3.069      0.002       1.378       6.248
e              0.0048      0.009      0.539      0.590      -0.013       0.022
s             -0.0059      0.011     -0.520      0.603      -0.028       0.016
g             -0.0249      0.015     -1.703      0.089      -0.054       0.004
==============================================================================
"""

In [26]:
df2.logit=smf.glm('ret ~ e+s+g+AV', data=df2, family=sm.families.Binomial()).fit()
df2.logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    ret   No. Observations:                  559
Model:                            GLM   Df Residuals:                      555
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -297.39
Date:                Thu, 20 May 2021   Deviance:                       594.78
Time:                        21:57:11   Pearson chi2:                     551.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1623      1.344      3.842      0.000       2.529       7.796
e              0.0007      0.009      0.079      0.937      -0.017       0.019
s             -0.0099      0.012     -0.848      0.397      -0.033       0.013
g             -0.0323      0.015     -2.142      0.032      -0.062      -0.003
AV          7.218e-14   3.25e-14      2.222      0.026    8.51e-15    1.36e-13
==============================================================================
"""

In [27]:
df2.logit=smf.glm('ret ~ e+s+g+log_AV', data=df2, family=sm.families.Binomial()).fit()
df2.logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    ret   No. Observations:                  559
Model:                            GLM   Df Residuals:                      554
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -298.41
Date:                Thu, 20 May 2021   Deviance:                       596.82
Time:                        21:57:12   Pearson chi2:                     556.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1139      1.904      0.060      0.952      -3.618       3.846
e              0.0020      0.009      0.224      0.823      -0.016       0.020
s             -0.0169      0.012     -1.357      0.175      -0.041       0.008
g             -0.0324      0.015     -2.129      0.033      -0.062      -0.003
log_AV         0.2135      0.083      2.570      0.010       0.051       0.376
==============================================================================
"""

In [28]:
df2.logit=smf.glm('ret ~ esg', data=df2, family=sm.families.Binomial()).fit()
df2.logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    ret   No. Observations:                  561
Model:                            GLM   Df Residuals:                      559
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -302.89
Date:                Thu, 20 May 2021   Deviance:                       605.77
Time:                        21:57:12   Pearson chi2:                     561.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.4513      1.126      3.064      0.002       1.244       5.659
esg           -0.0233      0.012     -2.026      0.043      -0.046      -0.001
==============================================================================
"""

#### 임육빈, <비재무적 정보가 기업성과에 미치는 영향: ESG 점수를 중심으로>(2019)

- ESG점수와 S,G 점수는 당기 및 차기의 기업가치를 높이는 유용한 정보
- 반면, E점수는 영향이 나타나지 않음